# Custom Control Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/control_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

[EPyT-Flow](https://github.com/WaterFutures/EPyT-Flow) is available on [PyPI](https://pypi.org/project/epyt-flow/) and can be installed via `pip install epyt-flow`:

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
from epyt_flow.data.networks import load_net1
from epyt_flow.simulation import ScenarioSimulator
from epyt_flow.simulation import CustomControlModule, ScadaData
from epyt_flow.utils import to_seconds, volume_to_level
from epyt_flow.simulation.events import ActuatorConstants

A custom control module mimicing the control rules stated in Net1.inp -- note that all control modules must be derived from the [CustomControlModule](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.scada.html#epyt_flow.simulation.scada.custom_control.CustomControlModule) class:

In [ ]:
class MyControl(CustomControlModule):
    def __init__(self, **kwds):
        # Tank and pump ID
        self.__tank_id = "2"
        self.__pump_id = "9"

        # Tank diameter could be also obtained by calling epyt.epanet.getNodeTankData
        self.__tank_diameter = 50.5

        # Lower and upper threshold on tank level
        self.__lower_level_threshold = 110
        self.__upper_level_threshold = 140

        super().__init__(**kwds)

    def step(self, scada_data: ScadaData) -> None:
        # Retrieve current water level in the tank
        tank_volume = scada_data.get_data_tanks_water_volume([self.__tank_id])[0, 0]
        tank_level = volume_to_level(float(tank_volume), self.__tank_diameter)

        # Decide if pump has to be deactivated or re-activated
        if tank_level <= self.__lower_level_threshold:
            self.set_pump_status(self.__pump_id, ActuatorConstants.EN_OPEN)
        elif tank_level >= self.__upper_level_threshold:
            self.set_pump_status(self.__pump_id, ActuatorConstants.EN_CLOSED)

Create new simulation based on Net1:

In [ ]:
sim = ScenarioSimulator(scenario_config=load_net1(verbose=False))

Set simulation duration to two days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Monitor states of tank "2" and pump "9":

In [ ]:
sim.set_tank_sensors(sensor_locations=["2"])
sim.set_pump_state_sensors(sensor_locations=["9"])

Note that Net1.inp contains some simple controls. Remove all of them by calling [remove_all_simple_controls()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.remove_all_simple_controls):

In [ ]:
sim.remove_all_simple_controls()

Add our custom control module by calling [add_custom_control()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_custom_control):

In [ ]:
sim.add_custom_control(MyControl())

Run the simulation and show sensor readings over time:

In [ ]:
scada_data = sim.run_simulation()

In [ ]:
scada_data.plot_pumps_state()

In [ ]:
scada_data.plot_tanks_water_volume()

Do not forget to close the simulation!

In [ ]:
sim.close()